In [22]:
import geopandas as gpd
import pandas as pd
from platform import python_version

print("Versions :")
print(f" - python : {python_version()}")
print(f" - pandas : {pd.__version__}")
print(f" - geopandas : {gpd.__version__}")

Versions :
 - python : 3.10.5
 - pandas : 1.4.3
 - geopandas : 0.11.0


# **Quelle est la surface des espaces naturelles en Martinique ?** 🌳🌴


## Source des données

[OCS-GE - Geomartinique](https://www.geomartinique.fr/accueil/ressources/ocs_ge)
 * [Nomenclature](https://catalogue.geomartinique.fr/geonetwork/srv/api/records/13b935b1-ee4e-4f86-8a81-2bed6e40f8fe/attachments/Nomenclature_OCSGE_Martinique_VDEF.ods) (La Martinique à une nomenclature spécifiques : Mangrove, bananiers, etc.)
 * [Couche de données](https://catalogue.geomartinique.fr/geonetwork/srv/fre/catalog.search#/metadata/13b935b1-ee4e-4f86-8a81-2bed6e40f8fe)

## Ressources
* [L’occupation du sol à la loupe avec l'OCS GE](https://www.cerema.fr/fr/centre-ressources/newsletters/signture/signture-69-artificialisation-sols-sa-mesure/occupation-du-sol-loupe-ocs-ge#:~:text=L'OCS%20GE%20est%20une%20base%20de%20donn%C3%A9es%20vectorielle%20grande,du%20sol%20(17%20postes).)
* [Portail artificialisation du sol](https://artificialisation.developpement-durable.gouv.fr/bases-donnees/ocs-ge)
* [Carte des espaces naturels de Martinique au sens de l'état (Parc naturels, ZNIEF, etc.)](https://carto.geomartinique.fr/CartesStatiques/DEAL/Biodiversite/WEB--A4-EspacesNaturelsMartinique-2018.jpeg)

## Vérification de la projection native de la la couche spatiale de l'OCS-GE 🌎
*Note :*
  - La projection correspond bien à la projectiond e référence pour la Martinique : "RGAF09 / UTM zone 20N" ou "EPSG:5490".
  - Cette projection est une projection locale cartésienne qui est utilisable pour le calcul des surfaces.

In [23]:
PATH = "input/ocs-ge_shp/deal972_occupation_sol_ge_2017_v2_s.shp"
gdf = gpd.read_file(PATH, encoding='utf-8')
gdf.crs

<Derived Projected CRS: EPSG:5490>
Name: RGAF09 / UTM zone 20N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: French Antilles onshore and offshore west of 60°W - Guadeloupe (including Grande Terre, Basse Terre, Marie Galante, Les Saintes, Iles de la Petite Terre, La Desirade); Martinique; St Barthélemy; northern St Martin.
- bounds: (-63.66, 14.08, -60.0, 18.31)
Coordinate Operation:
- name: UTM zone 20N
- method: Transverse Mercator
Datum: Reseau Geodesique des Antilles Francaises 2009
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

## Traduction des code "Couverture du sol" et "Usage du sol" en labels 🏷️

**Note :** 
* Les codes/labels us et cs correspondent au niveaux de classification le plus fin disponnible

In [24]:
# Chargement du tableau de traduction des code en labels pour la couverture du sol (cs) et l'usage du sol (us)
code_cs = pd.read_csv("input/221120_OCSGE_CS_labels.csv", sep=';', encoding='utf-8')
code_us = pd.read_csv("input/221120_OCSGE_US_labels.csv", sep=';', encoding='utf-8')

# Jointure du tableau de la couche spatiale et du tableau contenant les labels de l'OCS-GE
gdf = pd.merge(gdf, code_cs, left_on='code_cs', right_on='code').rename({'label':'label_CS'}, axis=1).drop('code', axis=1)
gdf = pd.merge(gdf, code_us, left_on='code_us', right_on='code').rename({'label':'label_US'}, axis=1).drop('code', axis=1)

# Création de la colonne des surfaces en mètres carrés. Les surfaces sont arrondies au mètre carré près. 
gdf['area'] = gdf.area.round()

# 5 premières lignes du tableau
gdf.head()

,gid,id,code_cs,code_us,millesime,source,ossature,id_origine,code_or,geometry,label_CS,label_US,area
0,1,1,CS1.1.1.1,US1.1,2017,None,0.0,NC,NC,"POLYGON ((727567.750 1610187.276, 727567.770 1...",Zones bâties,Agriculture,251.0
1,2,2,CS1.1.1.1,US1.1,2017,None,0.0,NC,NC,"POLYGON ((695732.002 1632771.602, 695751.802 1...",Zones bâties,Agriculture,1766.0
2,3,3,CS1.1.1.1,US1.1,2017,None,0.0,NC,NC,"POLYGON ((696053.251 1627147.354, 696053.861 1...",Zones bâties,Agriculture,201.0
3,4,4,CS1.1.1.1,US1.1,2017,None,0.0,NC,NC,"POLYGON ((696054.051 1627062.104, 696051.101 1...",Zones bâties,Agriculture,2376.0
4,5,5,CS1.1.1.1,US1.1,2017,None,0.0,NC,NC,"POLYGON ((696799.001 1626255.394, 696775.401 1...",Zones bâties,Agriculture,611.0


## Classification des surfaces "Usage du sol" vs. "Couverture du sol" 🔎
### Calcul de la surface total de la martinique 🇲🇶

In [25]:
S_total = gdf.area.sum().sum().round()
print(f"""Surface totale de la martinique: 
      - {int(S_total)} m2
      - {int(S_total)/ 10000} ha
      """)

Surface totale de la martinique: 
      - 1109064415 m2
      - 110906.4415 ha
      


### Création du tableau croisé 🔄

**Explication du tableau de données**

* *En colonne* : l'usage du sol identifiée par son code et son label ;
* *En ligne* : l'couverture du sol identifiée par son code et son label ;
* *Par cellule* : le pourcentage de la surface de classe définie par son usage du sol vs. sa couverture du sol ;

*Exemple de lecture :* 25,66% de la surface totale de la Martinique est couverte de forêt moyennement humide ou humide avec un usage de sylviculture

In [26]:
# Pourcentage de la surface totale de la Martinique
percent_pivot = (gdf.pivot_table(
                    index=['code_cs','label_CS'],
                    columns=['code_us','label_US'],
                    values='area',
                    aggfunc=sum,
                    margins=True,
                    margins_name='TOTAL')
               .divide(S_total)
               )

# Mise en forme
df_style = (percent_pivot
            .fillna(0)
            .style.format({k:"{:.2%}" for k in percent_pivot.columns})
            .background_gradient(axis=None,
                                 vmin=0.01,
                                 vmax=0.5,
                                 cmap='viridis')
            )
df_style

,code_us,US1.1,US1.2,US1.3,US1.4,US2,US2.4.2,US2.4.3,US2.4.4,US235,US3,US4.1.1,US4.1.2,US4.1.3,US4.1.4,US4.2,US4.3,US5,US6.1,US6.2,US6.3,TOTAL
,label_US,Agriculture,Sylviculture,Activités d’extraction,Pêche et aquaculture,Production secondaire,Production d'énergie thermique,Production d'énergie biomasse,Production d'énergies renouvelables,Usage mixte,Production tertiaire,Réseaux de transport routier,Réseaux de transport ferré,Réseaux de transport aérien,Réseaux de transport fluvial et maritime,Services de logistique et de stockage,Réseaux d’utilité publique,Usage résidentiel,Zones en transition,Zones abandonnées,Sans usage,
code_cs,label_CS,,,,,,,,,,,,,,,,,,,,,
CS1.1.1.1,Zones bâties,0.07%,0.00%,0.00%,0.00%,0.14%,0.00%,0.00%,0.00%,2.92%,0.27%,0.00%,0.00%,0.01%,0.00%,0.00%,0.01%,0.80%,0.01%,0.00%,0.00%,4.23%
CS1.1.1.2,Zones non bâties,0.00%,0.00%,0.00%,0.00%,0.13%,0.01%,0.01%,0.00%,0.16%,0.21%,1.73%,0.00%,0.05%,0.05%,0.00%,0.01%,0.17%,0.00%,0.00%,0.00%,2.55%
CS1.1.2.1,Zones à matériaux minéraux,0.03%,0.00%,0.12%,0.00%,0.05%,0.00%,0.00%,0.00%,0.09%,0.02%,0.01%,0.00%,0.00%,0.00%,0.00%,0.01%,0.06%,0.07%,0.00%,0.00%,0.47%
CS1.1.2.2,Zones à autres matériaux composites,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.01%,0.00%,0.00%,0.00%,0.00%,0.01%
CS1.2.1,Sols nus,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.19%,0.19%
CS1.2.1.6,Etang bois sec,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.21%,0.21%
CS1.2.2,Surfaces d'eau,0.00%,0.00%,0.00%,0.01%,0.00%,0.00%,0.00%,0.00%,0.07%,0.00%,0.00%,0.00%,0.00%,0.10%,0.00%,0.00%,0.00%,0.00%,0.00%,0.38%,0.58%
CS2.1.1.1,Peuplement de feuillus,0.43%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.43%


### Création d'un tableau en ligne 
**Note :** Même calcul mais usage plus facile pour analyse de donnée

In [27]:
percent_long = gdf.groupby(['code_us','label_US', 'code_cs','label_CS'])['area'].sum().divide(S_total)

## Sauvegarde

In [28]:
percent_pivot.to_csv('output/OCSGE_Martinique_pivot.csv', encoding='utf-8', sep=';')
percent_long.to_csv('output/OCSGE_Martinique_long.csv', encoding='utf-8', sep=';')